# Experimento TCC

## Libs

In [45]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis, KNeighborsClassifier
from psutil import cpu_count
from tqdm import trange
import numpy as np
import pandas as pd

In [34]:
from modules.classNCA import NCA
from modules.optimization_funcs import PSO, Gradient
from modules.utils import nca_pso_folds_run, nca_gradient_folds_run, knn_folds_run, process_results

In [35]:
dtypes = {
    'tau1': float, 'tau2': float, 'tau3': float, 'tau4': float,
    'p1': float, 'p2': float,'p3': float, 'p4': float, 
    'g1': float, 'g2': float, 'g3': float, 'g4': float}

EletricalGrid = pd.read_csv('dataset/Data_for_UCI_named.csv',
                            dtype=dtypes, nrows=300).drop('stab', axis=1)

In [36]:
X, y = EletricalGrid.drop('stabf',axis=1), EletricalGrid.stabf 

In [37]:
K_Folds = 5
K_neighbors = 5 # 50
max_iter= 10#1_000

In [43]:
nca_pso_options = {
    'X':X, 'y':y, 'K_Folds':K_Folds, 'model_class':NCA,
    'model_options':{
        'optimization_func':PSO, 'max_iter':max_iter, 
        'K_Folds': K_Folds,'k':K_neighbors, 
        'cpu_count':cpu_count(),'swarm_size':20
    }
}

nca_gradient_options = {
    'X': np.asarray(X), 'y': np.asarray(y), 'K_Folds': K_Folds,
    'k': K_neighbors, 'model_class': NeighborhoodComponentsAnalysis,
    'model_options': {'n_components':None, 'max_iter':max_iter,'random_state':42}
}

knn_options = {
    'X': np.asarray(X), 'y': np.asarray(y), 'K_Folds': K_Folds,'k': None, 
    'model_options': {'n_neighbors': K_neighbors, 'n_jobs': cpu_count()},
    'model_class': KNeighborsClassifier
}

In [ ]:
print('Execução de validação cruzada para NCA+PSO, NCA+Gradient e KNN')

In [13]:
nca_pso_results = [nca_pso_folds_run(**nca_pso_options) for _ in trange(10)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [04:00<00:00, 24.04s/it]


In [50]:
nca_gradient_results = [nca_gradient_folds_run(**nca_gradient_options) for _ in trange(10, position=0,leave=True)]

  0%|                                                                                                                                   | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.30it/s]
2it [00:01,  1.30it/s]
3it [00:02,  1.30it/s]
4it [00:03,  1.30it/s]
5it [00:03,  1.30it/s]
 10%|████████████▎                                                                                                              | 1/10 [00:03<00:34,  3.86s/it]
0it [00:00, ?it/s]
1it [00:00,  1.30it/s]
2it [00:01,  1.30it/s]
3it [00:02,  1.29it/s]
4it [00:03,  1.30it/s]
5it [00:03,  1.30it/s]
 20%|████████████████████████▌                                                                                                  | 2/10 [00:07<00:30,  3.87s/it]
0it [00:00, ?it/s]
1it [00:00,  1.31it/s]
2it [00:01,  1.31it/s]
3it [00:02,  1.30it/s]
4it [00:03,  1.30it/s]
5it [00:03,  1.30it/s]
 30%|████████████████████████████████████▉                                                                            

In [ ]:
knn_results = [knn_folds_run(**knn_options) for _ in trange(10)]

In [26]:
nca_pso_processed_results = process_results(nca_pso_results, K_Folds, 'NCA+PSO')
nca_gradient_processed_results = process_results(nca_gradient_based_results, K_Folds, 'NCA+Gradient')
knn_processed_results = process_results(knn_results, K_Folds, 'KNN')

In [29]:
df = pd.concat(
    [
        pd.DataFrame(nca_pso_processed_results), 
        pd.DataFrame(nca_gradient_processed_results),
        pd.DataFrame(knn_processed_results)
    ]).set_index('Model')

In [30]:
df

,Mean Time,Std Time,Mean Acc,Std Acc,Mean Mcc,Std Mcc
Model,,,,,,
NCA+PSO,24.030382,0.323351,0.789333,0.060016,0.536392,0.128498
NCA+Gradient,3.773018,0.009341,0.800000,0.052705,0.560192,0.109664
